In [35]:
import pandas as pd
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('data/api_metod_40k.csv')

In [3]:
df.head()

,Title,id,price,author,publisher,pub_year,s_tile
0,Learning Go,211190367,71.990000,Jon Bodner,O'Reilly Media,2024,NaN
1,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,NaN
2,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,NaN
3,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,"The Big Ideas Behind Reliable, Scalable, and M..."
4,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,NaN


In [6]:
df.rename(columns={"s_tile":"description"}, inplace=1)

In [7]:
df.head()

,Title,id,price,author,publisher,pub_year,description
0,Learning Go,211190367,71.990000,Jon Bodner,O'Reilly Media,2024,NaN
1,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,NaN
2,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,NaN
3,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,"The Big Ideas Behind Reliable, Scalable, and M..."
4,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,NaN


In [12]:
df.dtypes

Title           object
id               int64
price          float64
author          object
publisher       object
pub_year         int64
description     object
dtype: object

In [18]:
# total percentage of empty decription columns.
df['description'].count()/df['description'].isna().count()*100

66.78669667416854

In [19]:
# generating a column with combined text of title and description only where there is no null value in description.
df['combined_text'] = df.apply(lambda x: x['Title']+' '+x['description'] if pd.notnull(x['description']) else np.nan, axis = 1)

In [20]:
df.head()

,Title,id,price,author,publisher,pub_year,description,combined_text
0,Learning Go,211190367,71.990000,Jon Bodner,O'Reilly Media,2024,NaN,NaN
1,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,NaN,NaN
2,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,NaN,NaN
3,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,"The Big Ideas Behind Reliable, Scalable, and M...",Designing Data-Intensive Applications The Big ...
4,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,NaN,NaN


In [21]:
df.shape

(39990, 8)

In [23]:
# creating a new df without original description.
df_with_new_description = df.drop(['description'], axis=1)

In [25]:
df_with_new_description.head()

,Title,id,price,author,publisher,pub_year,combined_text
0,Learning Go,211190367,71.990000,Jon Bodner,O'Reilly Media,2024,NaN
1,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,NaN
2,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,NaN
3,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,Designing Data-Intensive Applications The Big ...
4,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,NaN


In [26]:
df_with_new_description.shape

(39990, 7)

### creating a function to retrieve the book informaiton from the website ebooks.com

In [59]:
import requests
from bs4 import BeautifulSoup

def get_description(title, author):
    # URL encode the title and author for the query
    search_query = f"{title} {author}".replace(' ', '%20')
    

    api_url = f"https://www.ebooks.com/api/search/?term={search_query}&CountryCode=CA"
    

    headers = {
        'User-Agent': 'Your User Agent',
        'X-Requested-With': 'XMLHttpRequest'  # This header is often used for AJAX calls
    }

    # Make the API request
    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        data = response.json()

        book_detail_url = f"https://www.ebooks.com{data['books'][0]['book_url']}"
        

        response = requests.get(book_detail_url, headers=headers)
        if response.status_code != 200:
            return "Failed to retrieve the first book detail page"

x 
        soup = BeautifulSoup(response.content, 'html.parser')
        description_div = soup.find('div', id='description')
        if description_div:
            return description_div.get_text(strip=True)
        else:
            return "Description not found for the first book"
    else:
        return "Failed to retrieve search results from API"

"""
# Example usage
title = "Designing Data-Intensive Applications	"
author = "JMartin Kleppmann"
description = get_description(title, author)
print(description)
"""    

'\n# Example usage\ntitle = "Designing Data-Intensive Applications\t"\nauthor = "JMartin Kleppmann"\ndescription = get_description(title, author)\nprint(description)\n'

In [60]:
descriptions = []
for index, row in df_with_new_description.head(20).iterrows():
    description = get_first_search_result_description(row['Title'], row['author'])
    descriptions.append(description)
    #print(f"Book: {row['Title']} by {row['author']}, Description: {description}")

# Now, you can add the descriptions to the DataFrame or use them as needed
df_with_new_description['description'][:20] = descriptions

Book: Learning Go by Jon Bodner, Description: Go has rapidly become the preferred language for building web services. Plenty of tutorials are available to teach Go's syntax to developers with experience in other programming languages, but tutorials aren't enough. They don't teach Go's idioms, so developers end up recreating patterns that don't make sense in a Go context. This practical guide provides the essential background you need to write clear and idiomatic Go.No matter your level of experience, you'll learn how to think like a Go developer. Author Jon Bodner introduces the design patterns experienced Go developers have adopted and explores the rationale for using them. This updated edition also shows you how Go's generics support fits into the language.This book helps you:Write idiomatic code in Go and design a Go projectUnderstand the reasons behind Go's design decisionsSet up a Go development environment for a solo developer or teamLearn how and when to use reflection, unsafe, 

Book: The Staff Engineer's Path by Tanya Reilly, Description: For years, companies have rewarded their most effective engineers with management positions. But treating management as the default path for an engineer with leadership ability doesn't serve the industry well--or the engineer. The staff engineer's path allows engineers to contribute at a high level as role models, driving big projects, determining technical strategy, and raising everyone's skills.This in-depth book shows you how to understand your role, manage your time, master strategic thinking, and set the standard for technical work. You'll read about how to be a leader without direct authority, how to plan ahead to make the right technical decisions, and how to make everyone around you better, while still growing as an expert in your domain.By exploring the three pillars of a staff engineer's job, Tanya Reilly, a veteran of the staff engineer track, shows you how to:Take a broad, strategic view when thinking about your 

Book: Learning Domain-Driven Design by Vlad Khononov, Description: Building software is harder than ever. As a developer, you not only have to chase ever-changing technological trends but also need to understand the business domains behind the software. This practical book provides you with a set of core patterns, principles, and practices for analyzing business domains, understanding business strategy, and, most importantly, aligning software design with its business needs.Author Vlad Khononov shows you how these practices lead to robust implementation of business logic and help to future-proof software design and architecture. You'll examine the relationship between domain-driven design (DDD) and other methodologies to ensure you make architectural decisions that meet business requirements. You'll also explore the real-life story of implementing DDD in a startup company.With this book, you'll learn how to:Analyze a company's business domain to learn how the system you're building fit

/var/folders/5p/j3w3hx_s0rl_lc0q8wgy8jbr0000gn/T/ipykernel_7840/1941842621.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_new_description['combined_text'][:20] = descriptions


In [62]:
df_with_new_description.head(25)

,Title,id,price,author,publisher,pub_year,combined_text
0,Learning Go,211190367,71.990000,Jon Bodner,O'Reilly Media,2024,Go has rapidly become the preferred language f...
1,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,Tidying up messy software is a must. And that ...
2,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,"Through a recent series of breakthroughs, deep..."
3,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,Data is at the center of many challenges in sy...
4,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,Prepare for Microsoft Exam MS-102 and help dem...
5,Exam Ref MD-102 Microsoft Endpoint Administrator,210901632,53.862731,Andrew Warren,Pearson Education,2023,Prepare for Microsoft Exam MD-102—and help dem...
6,The Staff Engineer's Path,210670147,42.990000,Tanya Reilly,O'Reilly Media,2022,"For years, companies have rewarded their most ..."
7,Learning Web Design,96260103,67.990000,Jennifer Robbins,O'Reilly Media,2018,Do you want to build web pages but have no pri...
8,Deciphering Data Architectures,211219932,84.990000,James Serra,O'Reilly Media,2024,"Data fabric, data lakehouse, and data mesh hav..."
9,Fundamentals of Data Engineering,210595152,84.990000,Joe Reis,O'Reilly Media,2022,Data engineering has grown rapidly in the past...


In [63]:
# new function with progress report.
from tqdm import tqdm

def fetch_descriptions_for_range(df, start, end):
    # Check if start and end indices are within the bounds of the dataframe
    if start < 0 or end > len(df):
        return "Range is out of bounds of the dataframe"
    
    # Initialize a list to hold descriptions
    descriptions = []

    # Use tqdm for a progress bar
    for index, row in tqdm(df.iloc[start:end].iterrows(), total=end-start, desc="Fetching descriptions"):
        description = get_first_search_result_description(row['Title'], row['author'])
        # Save the description to the 'combined_text' column
        df.at[index, 'combined_text'] = description
        # Print out the progress
        print(f"Completed: {index+1-start} of {end-start} - '{row['Title']}' by {row['author']}")

    return df

# Call the function with the desired range
# Example usage to process books from index 1 to 20
df_with_new_description = fetch_descriptions_for_range(df_with_new_description, 40, 100)


Fetching descriptions:   5%|█                    | 1/19 [00:03<00:57,  3.18s/it]

Completed: 1 of 19 - 'Laws of UX' by Jon Yablonski


Fetching descriptions:  11%|██▏                  | 2/19 [00:06<00:53,  3.17s/it]

Completed: 2 of 19 - 'Database Internals' by Alex Petrov


Fetching descriptions:  16%|███▎                 | 3/19 [00:08<00:46,  2.91s/it]

...

Completed: 16 of 19 - 'Head First Python' by Paul Barry


Fetching descriptions:  89%|█████████████████▉  | 17/19 [00:55<00:06,  3.37s/it]

Completed: 17 of 19 - 'TypeScript Cookbook' by Stefan Baumgartner
Completed: 18 of 19 - 'C# 12 Pocket Reference' by Joseph Albahari


Fetching descriptions: 100%|████████████████████| 19/19 [00:58<00:00,  3.07s/it]

Completed: 19 of 19 - 'Discussing Design' by Adam Connor


In [67]:
df_with_new_description.head(41)

,Title,id,price,author,publisher,pub_year,combined_text
0,Learning Go,211190367,71.990000,Jon Bodner,O'Reilly Media,2024,Go has rapidly become the preferred language f...
1,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,Tidying up messy software is a must. And that ...
2,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,"Through a recent series of breakthroughs, deep..."
3,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,Data is at the center of many challenges in sy...
4,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,Prepare for Microsoft Exam MS-102 and help dem...
5,Exam Ref MD-102 Microsoft Endpoint Administrator,210901632,53.862731,Andrew Warren,Pearson Education,2023,Prepare for Microsoft Exam MD-102—and help dem...
6,The Staff Engineer's Path,210670147,42.990000,Tanya Reilly,O'Reilly Media,2022,"For years, companies have rewarded their most ..."
7,Learning Web Design,96260103,67.990000,Jennifer Robbins,O'Reilly Media,2018,Do you want to build web pages but have no pri...
8,Deciphering Data Architectures,211219932,84.990000,James Serra,O'Reilly Media,2024,"Data fabric, data lakehouse, and data mesh hav..."
9,Fundamentals of Data Engineering,210595152,84.990000,Joe Reis,O'Reilly Media,2022,Data engineering has grown rapidly in the past...


In [68]:
df_with_new_description.shape

(39990, 7)

In [88]:
# updated code with single line output
def fetch_descriptions_for_range(df, start_index, end_index):
    if start_index < 0 or end_index > len(df):
        return "Range is out of bounds of the dataframe"

    # Calculate the total number of books to process for display in the progress
    total_books = end_index - start_index

    for index in range(start_index, end_index):
        row = df.iloc[index]
        title = row['Title']
        author = row['author']
        description = get_first_search_result_description(title, author)
        df.at[index, 'combined_text'] = description

        # Print progress: 'Book x of y'
        print(f'\rBook {index} of {end_index}: "{title}" by {author} description fetched', end='', flush=True)
        sleep(0.5) 


    print()

    return df

"""# Example usage
start_index = 100  # Define the start index of the range
end_index = 500    # Define the end index of the range
df_with_new_description = fetch_descriptions_for_range(df_with_new_description, start_index, end_index)

"""

'# Example usage\nstart_index = 100  # Define the start index of the range\nend_index = 500    # Define the end index of the range\ndf_with_new_description = fetch_descriptions_for_range(df_with_new_description, start_index, end_index)\n\n'

In [100]:
# Example usage
start_index = 10000  # Define the start index of the range
end_index = 11000    # Define the end index of the range
df_with_new_description = fetch_descriptions_for_range(df_with_new_description, start_index, end_index)

Book 10999 of 11000: "Machine Learning in Bioinformatics" by Yanqing Zhang description fetchedli Talbi description fetchedededtchedinnekamp description fetchedne Learning" by Jean H Gallier description fetched


In [103]:
selected_rows = df_with_new_description.iloc[1:11000]  # .iloc is inclusive of the start index, but exclusive of the end index

# Save the selected rows to a CSV file
file_path = 'df_with_11kdesc.csv'  # Adjust the file path as needed
selected_rows.to_csv(file_path, index=False)

In [94]:
df_with_new_description.shape

(39990, 7)

In [104]:
df_with_new_description.head()

,Title,id,price,author,publisher,pub_year,combined_text
0,Learning Go,211190367,71.990000,Jon Bodner,O'Reilly Media,2024,Go has rapidly become the preferred language f...
1,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,Tidying up messy software is a must. And that ...
2,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,"Through a recent series of breakthroughs, deep..."
3,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,Data is at the center of many challenges in sy...
4,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,Prepare for Microsoft Exam MS-102 and help dem...


In [111]:
df_with_new_description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39990 entries, 0 to 39989
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          39990 non-null  object 
 1   id             39990 non-null  int64  
 2   price          39879 non-null  float64
 3   author         39976 non-null  object 
 4   publisher      39990 non-null  object 
 5   pub_year       39990 non-null  int64  
 6   combined_text  31171 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 2.1+ MB


In [112]:
df_with_new_description['description'] = df_with_new_description['combined_text'].astype(str)

In [113]:
df_with_new_description.head()

,Title,id,price,author,publisher,pub_year,combined_text,description
0,Learning Go,211190367,71.990000,Jon Bodner,O'Reilly Media,2024,Go has rapidly become the preferred language f...,Go has rapidly become the preferred language f...
1,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,Tidying up messy software is a must. And that ...,Tidying up messy software is a must. And that ...
2,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,"Through a recent series of breakthroughs, deep...","Through a recent series of breakthroughs, deep..."
3,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,Data is at the center of many challenges in sy...,Data is at the center of many challenges in sy...
4,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,Prepare for Microsoft Exam MS-102 and help dem...,Prepare for Microsoft Exam MS-102 and help dem...


In [115]:
df_with_new_description = df_with_new_description.drop('combined_text', axis=1)


In [127]:
df_with_new_description.head()

,Title,id,price,author,publisher,pub_year,description
0,Learning Go,211190367,71.990000,Jon Bodner,O'Reilly Media,2024,Go has rapidly become the preferred language f...
1,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,Tidying up messy software is a must. And that ...
2,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,"Through a recent series of breakthroughs, deep..."
3,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,Data is at the center of many challenges in sy...
4,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,Prepare for Microsoft Exam MS-102 and help dem...


In [128]:
df_with_new_description['description'] = df_with_new_description['description'].astype(str)

In [130]:
df_with_new_description["description"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 39990 entries, 0 to 39989
Series name: description
Non-Null Count  Dtype 
--------------  ----- 
39990 non-null  object
dtypes: object(1)
memory usage: 312.6+ KB


In [131]:
df_with_new_description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39990 entries, 0 to 39989
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        39990 non-null  object 
 1   id           39990 non-null  int64  
 2   price        39879 non-null  float64
 3   author       39976 non-null  object 
 4   publisher    39990 non-null  object 
 5   pub_year     39990 non-null  int64  
 6   description  39990 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 2.1+ MB


In [52]:
df11k = pd.read_csv("df_with_11kdesc.csv")

In [53]:
df11k.head()

,Title,id,price,author,publisher,pub_year,description
0,Tidy First?,211127822,42.990000,Kent Beck,O'Reilly Media,2023,Tidying up messy software is a must. And that ...
1,"Hands-On Machine Learning with Scikit-Learn, K...",210681725,96.990000,Aurélien Géron,O'Reilly Media,2022,"Through a recent series of breakthroughs, deep..."
2,Designing Data-Intensive Applications,95729334,67.990000,Martin Kleppmann,O'Reilly Media,2017,Data is at the center of many challenges in sy...
3,Exam Ref MS-102 Microsoft 365 Administrator,210964419,53.862731,Orin Thomas,Pearson Education,2023,Prepare for Microsoft Exam MS-102 and help dem...
4,Exam Ref MD-102 Microsoft Endpoint Administrator,210901632,53.862731,Andrew Warren,Pearson Education,2023,Prepare for Microsoft Exam MD-102—and help dem...


In [54]:
df11k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        10999 non-null  object 
 1   id           10999 non-null  int64  
 2   price        10973 non-null  float64
 3   author       10995 non-null  object 
 4   publisher    10999 non-null  object 
 5   pub_year     10999 non-null  int64  
 6   description  10996 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 601.6+ KB


In [55]:
df11k['description'] = df11k['description'].astype(str)

In [56]:
df11k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        10999 non-null  object 
 1   id           10999 non-null  int64  
 2   price        10973 non-null  float64
 3   author       10995 non-null  object 
 4   publisher    10999 non-null  object 
 5   pub_year     10999 non-null  int64  
 6   description  10999 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 601.6+ KB


In [60]:
# Count the occurrence of the specific texts and empty strings in the 'combined_text' column
count_failed_api = (df11k['description'] == "Failed to retrieve search results from API").sum()
count_no_description = (df11k['description'] == "The publisher of this book has not provided a description. Please check back later.").sum()
count_empty = (df11k['description'] == "").sum()  # Use .isna().sum() if you want to count NaN values instead

# Print the counts
print("Count of 'Failed to retrieve search results from API':", count_failed_api)
print("Count of 'The publisher of this book has not provided a description. Please check back later.':", count_no_description)
print("Count of empty strings in 'description':", count_empty)


Count of 'Failed to retrieve search results from API': 195
Count of 'The publisher of this book has not provided a description. Please check back later.': 1805
Count of empty strings in 'description': 0


In [61]:
# Filter rows that match the given conditions
condition = (df11k['description'] != "Failed to retrieve search results from API") & \
            (df11k['description'] != "The publisher of this book has not provided a description. Please check back later.") & \
            (df11k['description'] != "")

# Apply the condition to the DataFrame and drop the rows that don't meet the criteria
df11k_filtered = df11k[condition]

# If you want to remove rows with NaN values as well
df11k_filtered = df11k_filtered.dropna(subset=['description'])

In [62]:
df11k_filtered.shape

(8999, 7)

In [63]:
# saving the 11k filtered record to csv file
df11k_filtered.to_csv('data/df11k_filtered.csv', index=False)